# Import Libraries

In [ ]:
from __future__ import print_function
import os, sys, math
import matplotlib.pyplot as plt
root = os.path.abspath(os.getcwd())
libs = root + '/libs'
data = root + '/data'
cobras = root + '/cobras_ts'
sys.path.append(libs)
sys.path.append(data)
sys.path.append(cobras)
from nn_utils import *
from utilsData import *
from utilsData import read_dataset
from utilsData import transform_labels
import keras
from keras.utils import plot_model
from keras.models import Sequential, load_model
from keras.models import Model
from keras.datasets import mnist
from keras.layers import *
from keras.utils import to_categorical
import pandas as pd
import json
import numpy as np
import ctypes
import svn
import svn.remote
import svn.local
import sklearn 
%matplotlib inline
from sqlalchemy import create_engine
import plotly.graph_objects as go
import time
from IPython import display
from dtaidistance import dtw

from cobras_dtw import COBRAS_DTW
from notebookquerier_ts import NotebookQuerierTS

# Define Constants and Variables

In [ ]:
UNSIGN_TO_SIGN_SHIFT = 128
# Define data
listx_test            = []
listx_train           = []
listy_test            = []
listy_train           = []
listAdaptedy_test     = []
listAdaptedy_train    = []
defaultY              = [1,0,0,0]
adaptedY              = [1,0,0,0,0]

dataRoot              = "data"
dataset_name          = 'Cloth'
rootWithSlash         = dataRoot + '/' 
dataTest              = dataset_name + "_TEST.txt"
dataTrain             = dataset_name + "_TRAIN.txt"

svnRepoMagChannel     = "http://svn.hamilton.ch/svn/fw_XRP/branches/XMC4xxx/MagChannel/XRP_WM_T_2_0X_00_0007/Source"
svnRepoCheckoutDest   = "D:/tempTrunk/"

pickleObjectNameW      = 'lowstandardhighvolumetipswater.pkl'
pickleObjectNameG      = 'lowstandardhighvolumetipsglycerin.pkl'

weightsMagChannel     = "../../../Source/NN/inc"
weightsMagChannelAbs  = os.path.abspath(weightsMagChannel) + '/'

INTERPOLATION         = 4
NUMBER_OF_SAMPLES_CNN = 250
ADM_LOWER_LIMIT_INT_8 = 127
ADM_UPPER_LIMIT_INT_8 = ADM_LOWER_LIMIT_INT_8

budget                = 100
alpha                 = 0.5
window                = 10

# Load Acquired Aspiration Data From SQL

In [ ]:
SQL_USER              = 'root'
SQL_PASSWORT          = 'C00lman.9'
localhost             = 'localhost'
SQL_PORT              = '3306'
databaseName          = 'phänomendetection'
datasheetName         = 'asp_cloth_air_maggain20highvolumetip'

db_connection_str     = 'mysql+pymysql://'+ SQL_USER + ':' + SQL_PASSWORT+'@' + localhost +':' + SQL_PORT + '/' + databaseName

#sql_DF                = pd.read_sql_table(datasheetName, con=db_connection_str)
sql_DF                = pd.read_pickle(pickleObjectNameW)
#sql_DF                = sql_DF.drop([52])
#sql_DF                = sql_DF.drop([69])
#sql_DF                = sql_DF.drop([115])

#SQL_USER              = 'root'
#SQL_PASSWORT          = 'C00lman.9'
#localhost             = 'localhost'
#SQL_PORT              = '3306'
#databaseName          = 'phänomendetection'
#datasheetName         = 'asp_cloth_air_maggain20'
#
#db_connection_str     = 'mysql+pymysql://'+ SQL_USER + ':' + SQL_PASSWORT+'@' + localhost +':' + SQL_PORT + '/' + databaseName
#
#sql_DF_2              = pd.read_sql_table(datasheetName, con=db_connection_str)
sql_DF_2               = pd.read_pickle(pickleObjectNameG)
#sql_DF_2              = sql_DF_2.drop([19])
#sql_DF_2              = sql_DF_2.drop([60])
#sql_DF_2              = sql_DF_2.drop([116])
#
#sql_DF                = sql_DF.append(sql_DF_2)
sql_DF                = pd.concat([sql_DF_2, sql_DF])

# Select just pressure datas 
sql_DFFilteredAsp        = sql_DF.loc[sql_DF.loc[:,'J_Read Pressure Data Active'] == 1,:]
# Select just class ids and pressure datas 
sql_DFFilteredAspColumns = sql_DFFilteredAsp.iloc[:,20:]
sql_DFFilteredAspColumns['ClassId'] = sql_DFFilteredAsp.loc[:,'ClassId']
sql_DFFilteredAspColumns = sql_DFFilteredAspColumns.reset_index()
sql_DFFilteredAspColumns = sql_DFFilteredAspColumns.drop(['index'], axis=1)
#sql_DFFilteredAspColumns = sql_DFFilteredAspColumns.drop([288]) 
#sql_DFFilteredAspColumns = sql_DFFilteredAspColumns.drop([290]) 
#sql_DFFilteredAspColumns = sql_DFFilteredAspColumns.drop([258]) 

# Different Aspiration Curves Raw Data

In [ ]:
plt.close()
fig = go.Figure()

title = 'Different Aspiration Curves Raw Data'
# Add traces

for i in range(0, sql_DFFilteredAspColumns.shape[0]):
    # Normal Aspiration
    if int(sql_DFFilteredAspColumns.loc[i,'ClassId']) == 0:
        fig.add_trace(go.Scatter(x=np.arange(len(np.array(sql_DFFilteredAspColumns.iloc[i,20:]))), y=np.array(sql_DFFilteredAspColumns.iloc[i,20:]),name='Asp. Normal', 
                                 line=dict(color='rgb(0,0,0)', width=1)))
    # Air Aspiration
    elif int(sql_DFFilteredAspColumns.loc[i,'ClassId']) == 1:
        fig.add_trace(go.Scatter(x=np.arange(len(np.array(sql_DFFilteredAspColumns.iloc[i,20:]))), y=np.array(sql_DFFilteredAspColumns.iloc[i,20:]),name='Asp. Air', 
                                 line=dict(color='rgb(255,0,0)', width=1)))       
    # Cloth Aspiration
    elif int(sql_DFFilteredAspColumns.loc[i,'ClassId']) == 2:
        fig.add_trace(go.Scatter(x=np.arange(len(np.array(sql_DFFilteredAspColumns.iloc[i,20:]))), y=np.array(sql_DFFilteredAspColumns.iloc[i,20:]),name='Asp. Cloth', 
                                 line=dict(color='rgb(0,0,255)', width=1)))  
        
        
annotations = []        
        
annotations.append(dict(xref='paper', yref='paper', x=0.0, y=1.05,
                              xanchor='left', yanchor='bottom',
                              text=title,
                              font=dict(family='Arial',
                                        size=30,
                                        color='rgb(37,37,37)'),
                              showarrow=False))        

fig.update_layout(annotations=annotations,
                   yaxis_title='Pressure [pa]',
                   xaxis_title='Number of Samples [N] ')

fig.show()

# Scale Time Data to 250 Points

Two different pressure data cases are possible
 - Aspiration pressure curve contains less than 250 points -> upsampling case
 - Aspiration pressure curve contains more than 250 points -> Downsampling case
 
The idea for this problem is to transform a digital signal to a nearly continuous signal by interpolation and resample it such that the whole curve in time contains 250 samples.

# Linear Interpolation of Curve

In [ ]:
sql_DFFilteredAspColumnsCopy                = sql_DFFilteredAspColumns.copy()
sql_DFFilteredAspColumnsCopy                = sql_DFFilteredAspColumnsCopy.drop(['ClassId'], axis= 1)
newDataframe                                = sql_DFFilteredAspColumnsCopy
for interpol in range(0,INTERPOLATION):
    newDataframe                            = pd.DataFrame()
    for i in range(0, sql_DFFilteredAspColumns.shape[0]):
        sql_DFFilteredAspColumnsWithoutNan  = np.array(sql_DFFilteredAspColumnsCopy.iloc[i,:].dropna())
        dataNumberingArray                  = np.arange(len(sql_DFFilteredAspColumnsWithoutNan)) + 1
        zeroPaddedData                      = np.insert(sql_DFFilteredAspColumnsWithoutNan, dataNumberingArray, 0)
        x = 0
        while x < (len(zeroPaddedData) - 2):
            zeroPaddedData[x + 1]           = ((zeroPaddedData[x] - zeroPaddedData[x + 2]) / 2.0) + zeroPaddedData[x + 2]
            x = x + 2
    
        interpolatedData                    = pd.DataFrame(zeroPaddedData)
        interpolatedDataT                   = interpolatedData.T
        
        newDataframe                        = pd.concat([newDataframe, interpolatedDataT])
        
    newDataframe                            = newDataframe.reset_index()
    newDataframe                            = newDataframe.drop(['index'], axis=1)
                                                    
    sql_DFFilteredAspColumnsCopy            = newDataframe      


# Resample Curves Such That Wohle Curve Is Sampled Equally

In [ ]:
totalResampledData                          = pd.DataFrame()
for i in range(0, sql_DFFilteredAspColumns.shape[0]):
    sampleLengthInterpolatedCurve           = len(sql_DFFilteredAspColumnsCopy.iloc[i, :].dropna())
    numberOfOffsetSamples                   = sampleLengthInterpolatedCurve/NUMBER_OF_SAMPLES_CNN
    sampleIteration                         = 0
    newDataframeWithoutNan                  = sql_DFFilteredAspColumnsCopy.iloc[i, :].dropna()
    resampledData                           = []
    for k in range(0, NUMBER_OF_SAMPLES_CNN):
        resampledData.append(newDataframeWithoutNan[int(sampleIteration)])
        sampleIteration                    +=numberOfOffsetSamples
    
    resampledDataPd                         = pd.DataFrame(np.array(resampledData))
    totalResampledData                      = pd.concat([totalResampledData, resampledDataPd.T])
  
totalResampledData                          = totalResampledData.reset_index()
totalResampledData                          = totalResampledData.drop(['index'], axis=1)
sql_DFFilteredAspColumnsResIndex            = sql_DFFilteredAspColumns.reset_index()
sql_DFFilteredAspColumnsResIndex            = sql_DFFilteredAspColumnsResIndex.drop(['index'], axis=1)
totalResampledData['ClassId']               = list(sql_DFFilteredAspColumnsResIndex.loc[:,'ClassId'])
# Make sure ClassId stands at first position
totalResampledDataColumns                   = totalResampledData.columns.tolist()
totalResampledDataColumns                   = totalResampledDataColumns[-1:] + totalResampledDataColumns[:-1]
totalResampledData                          = totalResampledData[totalResampledDataColumns]

# Save to CSV 

In [ ]:
totalResampledData.to_csv(rootWithSlash+dataTest, index=False, header=False)
totalResampledData.to_csv(rootWithSlash+dataTrain, index=False, header=False)

# Cobras TS Automatic Clustering Algorithm As Data Verification 

In [ ]:
#data = np.loadtxt(rootWithSlash+dataTest, delimiter=',',skiprows=0)
#series = data[:,1:] # you should only do this if your first column are class labels, as they are in the UCR files
#series = series[:100,:] # taking only the first 10 series for example purposes

In [ ]:
### Constructing the pairwise affinity matrix
#dists = dtw.distance_matrix(series, window=int(0.01 * window * series.shape[1]))
#dists[dists == np.inf] = 0
#dists = dists + dists.T - np.diag(np.diag(dists))
#affinities = np.exp(-dists * alpha)

In [ ]:
### Starting the clustering process
### Queried pairs will be plotted in the notebook
### A must-link is given by entering 'Y' in the notebook prompt, a cannot-link by entering 'n'.
### Each time an intermediate clustering is shown, you can decide to stop clustering (by entering 'n' in the prompt),\
### or continue answering queries (by entering  'Y' in the prompt).
#clusterer = COBRAS_DTW(affinities, NotebookQuerierTS(data), budget)
#clustering, intermediate_clusterings, runtimes, ml, cl = clusterer.cluster()

In [ ]:
### After clustering, you can ask for the cluster labeling.
#print(clustering.construct_cluster_labeling())

In [ ]:
## Return new classification back to 
#data[0:len(clustering.construct_cluster_labeling()), 0] = clustering.construct_cluster_labeling()

In [ ]:
#np.savetxt(rootWithSlash+dataTest ,data, delimiter=',')

# Convert Test/Train Cap Level Json File to Dataframe and Numpy Array

In [ ]:
# Load text file

datasets_dict = read_dataset(dataRoot, dataset_name) 
x_train       = datasets_dict[dataset_name][0]
y_train       = datasets_dict[dataset_name][1]
x_test        = datasets_dict[dataset_name][2]
y_test        = datasets_dict[dataset_name][3]
y_trainClass  = datasets_dict[dataset_name][1]
y_testClass   = datasets_dict[dataset_name][3]

nb_classes = len(np.unique(np.concatenate((y_train,y_test),axis =0)))
# make the min to zero of labels
y_train,y_test = transform_labels(y_train,y_test)

# save orignal y because later we will use binary
y_true = y_test.astype(np.int64) 
# transform the labels from integers to one hot vectors
enc = sklearn.preprocessing.OneHotEncoder()
enc.fit(np.concatenate((y_train,y_test),axis =0).reshape(-1,1))
y_train = enc.transform(y_train.reshape(-1,1)).toarray()
y_test = enc.transform(y_test.reshape(-1,1)).toarray()

if len(x_train.shape) == 2: # if univariate 
    # add a dimension to make it multivariate with one dimension 
    x_train = x_train.reshape((x_train.shape[0],x_train.shape[1],1))
    x_test = x_test.reshape((x_test.shape[0],x_test.shape[1],1))

input_shape = x_train.shape[1:]

# Scale Aspiration Data

\begin{align}
\Delta_s = max(S(t)) - min(S(t)) \\
S_s(t) = \frac{\Delta_{s128}}{\Delta_s} \Bigg(S(t) - \bigg|\frac{|max(S(t))| - |min(S(t))|}{2}\bigg|\Bigg) \hspace{1cm} |max(S(t))| > |min(S(t))|\\
S_s(t) = \frac{\Delta_{s128}}{\Delta_s} \Bigg(S(t) + \bigg|\frac{|max(S(t))| - |min(S(t))|}{2}\bigg|\Bigg) \hspace{1cm} |min(S(t))| > |max(S(t))|
\end{align}

where $S_s(t)$ defines the scaled pressure data, $S(t)$ defines the unscaled pressure data, $\Delta_{s128}$ the max range of a int_8 datatype (255) and $\Delta_s$ the max range of the unscaled pressure data $S(t)$

In [ ]:
plt.close()
fig = go.Figure()

title = 'Different Aspiration Curves Test Data'
# Add traces

for i in range(0,x_test.shape[0]):
    # Normal Aspiration
    if int(y_testClass[i]) == 0:
        fig.add_trace(go.Scatter(x=np.arange(len(x_test[i])), y=x_test[i,:].ravel(),name='Asp. Normal', 
                                 line=dict(color='rgb(0,0,0)', width=1)))
    # Air Aspiration
    elif int(y_testClass[i]) == 1:
        fig.add_trace(go.Scatter(x=np.arange(len(x_test[i])), y=x_test[i,:].ravel(),name='Asp. Air', 
                                 line=dict(color='rgb(255,0,0)', width=1)))       
    # Cloth Aspiration
    elif int(y_testClass[i]) == 2:
        fig.add_trace(go.Scatter(x=np.arange(len(x_test[i])), y=x_test[i,:].ravel(),name='Asp. Cloth', 
                                 line=dict(color='rgb(0,0,255)', width=1)))  
        
        
annotations = []        
        
annotations.append(dict(xref='paper', yref='paper', x=0.0, y=1.05,
                              xanchor='left', yanchor='bottom',
                              text=title,
                              font=dict(family='Arial',
                                        size=30,
                                        color='rgb(37,37,37)'),
                              showarrow=False))        

fig.update_layout(annotations=annotations,
                   yaxis_title='Pressure [pa]',
                   xaxis_title='Number of Samples [N] ')

fig.show()

# Display Train X (time) and Y (Normal Aspiration, Cloth Aspiration)

In [ ]:
fig = go.Figure()
title = 'Different Aspiration Curves Train Data'
scaledTrain = x_train
# Add traces
for i in range(0,x_train.shape[0]):
    # Scale data
    temp        = []
    data        = x_train[i]
    delta_s     = max(data) - min(data)
    if(abs(max(data)) > abs(min(data))):
        tempScaled          = 253.0/delta_s * np.array(data) 
        scaledTrain[i,:]    = tempScaled + (ADM_UPPER_LIMIT_INT_8 - max(tempScaled))
    elif(abs(min(data)) > abs(max(data))): 
        tempScaled          = 253.0/delta_s * np.array(data) 
        scaledTrain[i,:]    = tempScaled + (-ADM_LOWER_LIMIT_INT_8 - min(tempScaled))
    
    # Normal Aspiration
    if int(y_trainClass[i]) == 0:
        fig.add_trace(go.Scatter(x=np.arange(len(x_train[i])), y=scaledTrain[i,:].ravel(),name='Asp. Normal', 
                                 line=dict(color='rgb(0,0,0)', width=1)))
    # Air Aspiration Aspiration
    elif int(y_trainClass[i]) == 1:
        fig.add_trace(go.Scatter(x=np.arange(len(x_train[i])), y=scaledTrain[i,:].ravel(),name='Asp. Air', 
                                 line=dict(color='rgb(255,0,0)', width=1)))       
    # Cloth Aspiration
    elif int(y_trainClass[i]) == 2 :
        fig.add_trace(go.Scatter(x=np.arange(len(x_train[i])), y=scaledTrain[i,:].ravel(),name='Asp. Cloth', 
                                 line=dict(color='rgb(0,0,255)', width=1)))  
        
annotations = []        
        
annotations.append(dict(xref='paper', yref='paper', x=0.0, y=1.0,
                              xanchor='left', yanchor='bottom',
                              text=title,
                              font=dict(family='Arial',
                                        size=30,
                                        color='rgb(37,37,37)'),
                              showarrow=False))        

fig.update_layout(annotations=annotations,
                   yaxis_title='Pressure [pa]',
                   xaxis_title='Number of Samples [N] ')
fig.show()


# Display Test X (time) and Y (Normal Aspiration, Cloth Aspiration)

In [ ]:
plt.close()
fig = go.Figure()

title = 'Different Aspiration Curves Test Data'
scaledTest = x_test
# Add traces
for i in range(0,x_test.shape[0]):
    # Scale data
    temp        = []
    data        = x_test[i]
    delta_s     = max(data) - min(data)
    if(abs(max(data)) > abs(min(data))):
        tempScaled          = 253.0/delta_s * np.array(data) 
        scaledTest[i,:]    = tempScaled + (ADM_UPPER_LIMIT_INT_8 - max(tempScaled))
    elif(abs(min(data)) > abs(max(data))): 
        tempScaled          = 253.0/delta_s * np.array(data) 
        scaledTest[i,:]    = tempScaled + (-ADM_LOWER_LIMIT_INT_8 - min(tempScaled))

for i in range(0,x_test.shape[0]):
    # Normal Aspiration
    if int(y_testClass[i]) == 0:
        fig.add_trace(go.Scatter(x=np.arange(len(x_test[i])), y=scaledTest[i,:].ravel(),name='Asp. Normal', 
                                 line=dict(color='rgb(0,0,0)', width=1)))
    # Air Aspiration
    elif int(y_testClass[i]) == 1:
        fig.add_trace(go.Scatter(x=np.arange(len(x_test[i])), y=scaledTest[i,:].ravel(),name='Asp. Air', 
                                 line=dict(color='rgb(255,0,0)', width=1)))       
    # Cloth Aspiration
    elif int(y_testClass[i]) == 2:
        fig.add_trace(go.Scatter(x=np.arange(len(x_test[i])), y=scaledTest[i,:].ravel(),name='Asp. Cloth', 
                                 line=dict(color='rgb(0,0,255)', width=1)))  
        
        
annotations = []        
        
annotations.append(dict(xref='paper', yref='paper', x=0.0, y=1.05,
                              xanchor='left', yanchor='bottom',
                              text=title,
                              font=dict(family='Arial',
                                        size=30,
                                        color='rgb(37,37,37)'),
                              showarrow=False))        

fig.update_layout(annotations=annotations,
                   yaxis_title='Pressure [pa]',
                   xaxis_title='Number of Samples [N] ')

fig.show()

# Define Layers of CNN Algorithm

In [ ]:
#https://adventuresinmachinelearning.com/keras-tutorial-cnn-11-lines/
def KModel(x_train, y_train, x_test, y_test):
    # Shuffle
    permutation = np.random.permutation(y_train.shape[0])
    x_train = x_train[permutation, :, :]
    y_train = y_train[permutation]
       
    inputs = Input(shape=x_train.shape[1:])
    # Layer Block 1
    x = Conv1D(32, kernel_size=(9), strides=(2), padding='same')(inputs)
    x = BatchNormalization()(x)
    x = Dropout(0.2)(x)
    x = ReLU()(x)
    x = MaxPool1D(2, strides=2)(x)

    # Layer Block 2
    x1 = Conv1D(32, kernel_size=(5), strides=(1), padding="same")(x)
    x1 = BatchNormalization()(x1)
    x1 = Dropout(0.2)(x1)
    x1 = ReLU()(x1)
    x1 = MaxPool1D(2, strides=2)(x1)

    # Layer Block 3
    x2 = Conv1D(32, kernel_size=(3), strides=(1), padding="same")(x)
    x2 = BatchNormalization()(x2)
    x2 = Dropout(0.2)(x2)
    x2 = ReLU()(x2)
    x2 = MaxPool1D(2, strides=2)(x2)

    # Layer Block 4
    x3 = MaxPool1D(2, strides=2)(x)
    x3 = Dropout(0.2)(x3)

    # concate all inception layers
    x = concatenate([ x1, x2,x3], axis=-1)

    # conclusion
    x = Conv1D(48, kernel_size=(3), strides=(1), padding="same")(x)
    x = ReLU()(x)
    x = MaxPool1D(2, strides=2)(x)
    x = Dropout(0.2)(x)

    # our netowrk is not that deep, so a hidden fully connected layer is introduce
    x = Flatten()(x)
    x = Dense(64)(x)
    x = Dropout(0.2)(x)
    x = ReLU()(x)
    x = Dense(3)(x)
    y = Softmax()(x)
    
    model = Model(inputs=inputs, outputs=y)
    model.compile(loss="categorical_crossentropy", optimizer="Adam", metrics=['accuracy'])
    model.fit(x_train, y_train, batch_size=16, epochs=100, shuffle=True, verbose=2, validation_data=(x_test, y_test))
    return model

# Train CNN Algorithm

In [ ]:
if os.path.exists('.shift_list'):
    os.remove(".shift_list")
model = KModel(scaledTrain, y_train, scaledTest, y_test)
model.save(dataRoot+'model.h5')

# Checkout SVN Repository

In [ ]:
# Start SVN Client
#client = svn.remote.RemoteClient(svnRepoMagChannel)
#client.checkout(svnRepoCheckoutDest)

# Generate C Code for NN Library and Transfer Weights File SVN

In [ ]:
#weights= svnRepoCheckoutDest + 'NN_weights.h'
weights= 'D:/ProjectClotDetection/MagChannelImplementation/branches/XRP_WM_T_2_0X_00_0008/Source/NN/' + 'NN_weights.h'
generate_model(model, x_test, name=weights)

# Commit SVN Repository

In [ ]:
#Committing the actuall changes
#r = svn.local.LocalClient(svnRepoCheckoutDest)
#r.commit('Updated weights of Mag Channel Phenomen Detection CNN algorithm')